In [52]:
import pandas as pd 
import numpy as np
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.manifold import Isomap
from keras.models import Sequential
from keras.layers import Dense

In [53]:
def load_data():
    dfs = []
    for filename in os.listdir('CSVs/'):
        year_df = pd.read_csv('CSVs/'+filename)
        year_df['winner'] = 1*(year_df['rank'] == 1)
        dfs.append(year_df)
    df = pd.concat(dfs)
    df.index = range(0,df.shape[0])
    return df

def process_data(df):
    
    # defining country mapping
    country_to_id_mapping = {}
    id_to_country_mapping = {}
    
    id_count = 0
    for code in df['code'].unique():
        country_to_id_mapping[code] = id_count
        id_to_country_mapping[id_count] = code
        
        id_count += 1
        
    # new data
    new_ids = []
    for code in df['code']:
        new_ids.append(country_to_id_mapping[code])
    df['country_id'] = new_ids
    
    df['top_5'] = (df['rank'] < 5)*1
    df['top_10'] = (df['rank'] < 10)*1
    df['top_15'] = (df['rank'] < 15)*1
    df['top_25'] = (df['rank'] < 25)*1
    df['top_50'] = (df['rank'] < 50)*1
    df['top_100'] = (df['rank'] < 100)*1
    
    # dropping columns
    df = df.drop(columns = ['problem1','problem2','problem3','problem4','problem5','problem6'])
    
    df['next_year_winner'] = 0
    new_dfs = []
    for year in df['year'].unique():
        if year < 2020:
            tmp1 = df[df['year'] == year + 1].copy()
            next_year_winner = tmp1[tmp1['rank'] == 1]['country_id'].iloc[0]
            new_df = df[df['year'] == year].copy()
            new_df['next_year_winner'] = (new_df['country_id'] == next_year_winner)*1
            
            for i in [5,10,15,25,50,100]:
                top_i = tmp1[tmp1['top_'+str(i)] == 1]
                countries = list(top_i['country_id'].unique())
                new_df['next_year_top_'+str(i)] = new_df['country_id'].isin(countries)*1
                
            
            new_dfs.append(new_df)
        else:
            new_df = df[df['year'] == 2020].copy()
            new_df['next_year_winner'] = 0
            new_dfs.append(new_df)
            
    df = pd.concat(new_dfs)
    
    dfs = []
    for code in df['code'].unique():
        code_df = df[df['code'] == code].sort_values(by = ['year'])
        code_df['total_wins'] = code_df['winner'].cumsum()
        code_df['total_gold'] = code_df['gold_medals'].cumsum()
        code_df['total_silver'] = code_df['silver_medals'].cumsum()
        code_df['total_bronze'] = code_df['bronze_medals'].cumsum()
        code_df['min_rank'] = code_df['rank'].cummin()
        code_df['max_rank'] = code_df['rank'].cummax()
        code_df['average_rank'] = code_df['rank'].cumsum()/np.arange(1,code_df.shape[0]+1)
        for i in [2,3,4]:
            code_df['average_rank_'+str(i)] = code_df['rank'].rolling(window = i).mean()
            code_df['recent_wins_'+str(i)] = code_df['rank'].rolling(window = i).sum()

        dfs.append(code_df)
    df = pd.concat(dfs).fillna(0)

    return df, country_to_id_mapping,id_to_country_mapping

In [54]:
df = load_data()
df, country_to_id_mapping,id_to_country_mapping = process_data(df)

In [55]:
features = ['total', 'rank','winner', 'country_id', 
            'top_5','top_10', 'top_15', 'top_25', 'top_50', 'top_100',
            'total_wins', 'min_rank',
            'max_rank', 'average_rank', 'average_rank_2', 'recent_wins_2',
            'average_rank_3', 'recent_wins_3', 'average_rank_4', 'recent_wins_4']

target = 'next_year_winner'

In [56]:
train, test = df[df['year'] <= 2019],  df[df['year'] == 2020]
X_train, y_train = train[features], train[target].astype(int)
X_test = test[features]

In [34]:
X_train.shape

(942, 20)

In [35]:
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=150, batch_size=10)

Epoch 1/150
95/95 [==============================] - 0s 976us/step - loss: 3.5717 - accuracy: 0.8620
Epoch 2/150
95/95 [==============================] - 0s 1ms/step - loss: 0.3616 - accuracy: 0.9777
Epoch 3/150
95/95 [==============================] - 0s 1ms/step - loss: 0.1464 - accuracy: 0.9904
Epoch 4/150
95/95 [==============================] - 0s 2ms/step - loss: 0.0503 - accuracy: 0.9883
Epoch 5/150
95/95 [==============================] - 0s 1ms/step - loss: 0.0354 - accuracy: 0.9904
Epoch 6/150
95/95 [==============================] - 0s 1ms/step - loss: 0.0270 - accuracy: 0.9894
Epoch 7/150
95/95 [==============================] - 0s 2ms/step - loss: 0.0324 - accuracy: 0.9862
Epoch 8/150
95/95 [==============================] - 0s 1ms/step - loss: 0.0303 - accuracy: 0.9883
Epoch 9/150
95/95 [==============================] - 0s 1ms/step - loss: 0.0310 - accuracy: 0.9862
Epoch 10/150
95/95 [==============================] - 0s 2ms/step - loss: 0.0269 - accuracy: 0.9883
Epoch 1

95/95 [==============================] - 0s 1ms/step - loss: 0.0211 - accuracy: 0.9915
Epoch 83/150
95/95 [==============================] - 0s 1ms/step - loss: 0.0346 - accuracy: 0.9883
Epoch 84/150
95/95 [==============================] - 0s 2ms/step - loss: 0.0247 - accuracy: 0.9873
Epoch 85/150
95/95 [==============================] - 0s 1ms/step - loss: 0.0262 - accuracy: 0.9873
Epoch 86/150
95/95 [==============================] - 0s 1ms/step - loss: 0.0232 - accuracy: 0.9904
Epoch 87/150
95/95 [==============================] - 0s 1ms/step - loss: 0.0265 - accuracy: 0.9894
Epoch 88/150
95/95 [==============================] - 0s 1ms/step - loss: 0.0374 - accuracy: 0.9883
Epoch 89/150
95/95 [==============================] - 0s 1ms/step - loss: 0.0229 - accuracy: 0.9894
Epoch 90/150
95/95 [==============================] - 0s 1ms/step - loss: 0.0221 - accuracy: 0.9904
Epoch 91/150
95/95 [==============================] - 0s 1ms/step - loss: 0.0247 - accuracy: 0.9894
Epoch 92/150


In [37]:
pred_probs = model.predict(X_test)
country_ids = X_test['country_id']
for_df = []
for i,j in zip(pred_probs, country_ids):
    win_prob = i
    country = id_to_country_mapping[j]
    odds = win_prob/(1-win_prob)
    for_df.append({'country':country,'prob':win_prob,'odds':odds})
results = pd.DataFrame(for_df)
results.to_csv('winning_odds.csv')

,country,prob,odds
1,USA,[0.35674542],[0.55459446]
0,CHN,[0.2452125],[0.3248762]
3,RUS,[0.14422467],[0.168531]
11,KOR,[0.07283309],[0.07855445]
4,THA,[0.010528684],[0.010640716]
2,SGP,[3.4686323e-07],[3.4686335e-07]
14,UKR,[3.2689397e-07],[3.2689405e-07]
13,POL,[1.3064596e-07],[1.3064597e-07]
10,JPN,[1.567338e-08],[1.567338e-08]
16,UNK,[9.543903e-09],[9.543903e-09]


In [47]:
from sklearn.model_selection import GridSearchCV
parameters = {'activation':["logistic", "relu"],
             'solver' : ['lbfgs','sgd','adam'],
             'alpha' : [0.00001,0.0001,0.001,0.01]}
nn = MLPClassifier(random_state = 42, max_iter = 1000)
clf = GridSearchCV(nn, parameters)
clf.fit(X_train, y_train)
optimal_params = clf.best_params_

c:\users\tyler\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\users\tyler\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\users\tyler\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.

c:\users\tyler\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\users\tyler\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\users\tyler\appdata\local\programs\python\python37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.

In [49]:
optimal_params

{'activation': 'logistic', 'alpha': 1e-05, 'solver': 'sgd'}

In [59]:
model = MLPClassifier()
model.fit(X_train, y_train)
model.score(X_train, y_train)
pred_probs = model.predict_proba(X_test)
country_ids = X_test['country_id']
for_df = []
for i,j in zip(pred_probs, country_ids):
    win_prob = i[1]
    country = id_to_country_mapping[j]
    odds = win_prob/(1-win_prob)
    for_df.append({'country':country,'prob':win_prob,'odds':odds})
results = pd.DataFrame(for_df)
results.sort_values(by = ['prob'], ascending = False).head(20)

,country,prob,odds
0,CHN,3.225284e-01,4.760767e-01
1,USA,3.129136e-01,4.554211e-01
3,RUS,2.055890e-01,2.587942e-01
11,KOR,4.468823e-02,4.677868e-02
4,THA,2.448860e-02,2.510334e-02
14,UKR,8.981567e-05,8.982374e-05
2,SGP,6.418087e-05,6.418498e-05
13,POL,5.638332e-05,5.638650e-05
16,UNK,2.993448e-06,2.993457e-06
22,AUS,3.984886e-07,3.984887e-07


In [60]:
model.score(X_train, y_train)

0.9904458598726115

In [61]:
nn_pipe = Pipeline([('pca',PCA()),('minmax',MinMaxScaler()),('nn', MLPClassifier())])
nn_pipe.fit(X_train, y_train)
pred_probs = nn_pipe.predict_proba(X_test)
country_ids = X_test['country_id']
for_df = []
for i,j in zip(pred_probs, country_ids):
    win_prob = i[1]
    country = id_to_country_mapping[j]
    odds = win_prob/(1-win_prob)
    for_df.append({'country':country,'prob':win_prob,'odds':odds})
results = pd.DataFrame(for_df)
results.sort_values(by = ['prob'], ascending = False).head(20)

,country,prob,odds
0,CHN,0.446413,0.806401
1,USA,0.322227,0.475421
11,KOR,0.139072,0.161538
3,RUS,0.114708,0.129571
4,THA,0.009416,0.009505
2,SGP,0.008185,0.008252
14,UKR,0.006699,0.006744
15,CAN,0.006185,0.006223
6,ROU,0.006072,0.006109
18,BRA,0.005784,0.005818


In [62]:
nn_pipe.score(X_train, y_train)

0.9904458598726115

In [46]:
for i in [5,10,15,25,50,100]:
    target = 'next_year_top_'+str(i)
    train, test = df[df['year'] <= 2019],  df[df['year'] == 2020]
    X_train, y_train = train[features], train[target].astype(int)
    X_test = test[features]

    model = MLPClassifier()
    model.fit(X_train, y_train)
    model.score(X_train, y_train)
    
    pred_probs = model.predict_proba(X_test)
    country_ids = X_test['country_id']
    for_df = []
    for i,j in zip(pred_probs, country_ids):
        win_prob = i[1]
        country = id_to_country_mapping[j]
        odds = win_prob/(1-win_prob)
        for_df.append({'country':country,'prob':win_prob,'odds':odds})
    results = pd.DataFrame(for_df)
    results.to_csv('odds_top_'+str(i))